In [1]:
## Specify file encoding type in python

# -*- coding: utf-8 -*-
print ("UTF-8 ENCODING")

UTF-8 ENCODING


In [2]:
import chardet # https://github.com/chardet/chardet
import glob
import codecs
import sys
import os
from TextLoader import *
from Hangulpy import *
print ("PACKAGES LOADED")

PACKAGES LOADED


In [3]:
## Convert UTF-8 encoded txt file

def conv_file(fromfile, tofile):
    with open(fromfile, "rb") as f:
        sample_text=f.read(10240)
    pred = chardet.detect(sample_text)
    if not pred['encoding'].upper() in ('EUC-KR', 'UTF-8', 'CP949', 'UTF-16LE'):
        print("WARNING! Unknown encoding! : %s = %s" % (fromfile, pred['encoding']))
        pred['encoding'] = "CP949" # 못찾으면 기본이 CP949
        formfile = fromfile + ".unknown"
    elif pred['confidence'] < 0.9:
        print ("WARNING! Unsured encofing! : %s = %s / %s")
        % (fromfile, pred['confidence'], pred['encoding'])
        formfile = fromfile + ".notsure"
    with codecs.open(fromfile, "r", encoding=pred['encoding'], errors="ignore") as f:
        with codecs.open(tofile, "w+", encoding="utf-8") as t:
            all_text = f.read()
            t.write(all_text)

In [4]:
## Generate UTF-8 encoded file

# SOURCE TXT FILE
fromfile = "./data/ninedreams.txt"
# TARGET TXT FILE
tofile   = "./data/ninedreams_utf8.txt"
conv_file(fromfile, tofile)
print ("UTF8-CONVERTING DONE")
print (" [%s] IS GENERATED" % (tofile))

UTF8-CONVERTING DONE
 [./data/ninedreams_utf8.txt] IS GENERATED


In [5]:
## Decompose Hangul (VERY IMPORTANT)

def dump_file(filename):
    result=u"" # <= UNICODE STRING 
    with codecs.open(filename,"r", encoding="UTF-8") as f:
        for line in f.readlines():
            line = tuple(line)
            result = result + decompose_text(line)
    return result
print ("FUNCTION READY")

FUNCTION READY


In [6]:
## For Python2 & Python3 compatibility...

if sys.version_info.major == 2:
    parsed_txt = dump_file(tofile).encode("utf8") 
else:
    parsed_txt = dump_file(tofile) 

print ("Parsing %s done" % (tofile))
# PRINT FIRST 100 CHARACTERS
print (parsed_txt[:100])

Parsing ./data/ninedreams_utf8.txt done


  ㅎㅏㄴᴥㄱㅜㄱᴥ ㄱㅜㄱᴥㅁㅜㄴᴥㅎㅏㄱᴥㅅㅏᴥㅅㅏㅇᴥ ㅇㅕㅇᴥㅇㅝㄴᴥㅎㅣᴥ ㅂㅣㅊᴥㄴㅏㄹᴥ ㅁㅕㅇᴥㅈㅏㄱᴥ
  ㄱㅜᴥㅇㅜㄴᴥㅁㅗㅇᴥ


  ㅈㅣᴥㅇㅡㄴᴥㅇㅣᴥ:ㄱㅣㅁᴥㅁㅏㄴᴥ


In [7]:
with open("data/input.txt", "w") as text_file:
    text_file.write(parsed_txt)
print ("Saved to a txt file")
print (text_file)

Saved to a txt file
<_io.TextIOWrapper name='data/input.txt' mode='w' encoding='UTF-8'>


In [8]:
## Compose hangul character from phoneme
data=[u'\u3147', u'\u3157', u'\u1d25', u'\u3134', u'\u3161', u'\u3139', u'\u1d25'
      , u' ', u'\u314f', u'\u3147', u'\u3145', u'\u314f', u'\u1d25', u'\u1d25'
      , u'\u3163', u'\u1d25', u' ', u'\u3147', u'\u1d25', u'\u3155', u'\u1d25'
      , u'\u3134', u'\u314f', u'\u1d25', u'\u3155', u'\u3147', u'\u1d25'
      , u'\u315b', u'\u3131', u'\u1d25', u'\u3147', u'\u3139', u'\u3146'
      , u'\u1d25', u'\u3137', u'\u314f', u'\u314e', u'\u3139', u'\u1d25'
      , u'\u3134', u'\u1d25', u'\u3145', u'\u3163', u'\u1d25', u'\u1d25'
      , u'\u314f', u'\u1d25', u'\u314e', u'\u314f', u'\u3147', u'\u3131'
      , u'\u3157', u'\u3134', u'\u1d25', u'\u1d25', u'\u315b', u'\u1d25'
      , u'\u3148', u'\u3153', u'\u3136', u'\u1d25', u' ', u'\u3145', u'\u3150'
      , u'\u3141', u'\u3136', u'\u3161', u'\u3134', u'\u3163', u'\u1d25', u'.'
      , u'\u3148', u'\u3153', u'\u3134', u'\u314e', u'\u3153', u'\u1d25', u'\u1d25'
      , u'\u3147', u'\u314f', u'\u3134', u'\u3148', u'\u314f', u'\u3139', u'\u315d'
      , u'\u314c', u'\u1d25', u'\u3161', u'\u3134', u'\u3148', u'\u3163', u'\u313a'
      , u'\u1d25', u' ', u'\u3147', u'\u3161', u'\u3146', u'\u1d25', u'?', u'\u3134'
      , u'\u1d25', u'\u314e', u'\u3163', u'\u1d25', u'\u3147', u'\u3148', u'\u314f'
      ]
print(automata("".join(data)))

오늘 ㅏㅇ사ㅣ ㅇㅕ나ㅕㅇㅛㄱㅇㄹㅆ닿ㄹㄴ시ㅏ항곤ㅛ젆 샘ㄶㅡ니.젆ㅓ앉ㅏ뤝ㅡㄴ짉 읐?ㄴ히ㅇ


In [9]:
## Generate "vocab.pkl" and "data.npy" in "data/" FROM "data/input.txt"
data_dir    = "data/"
batch_size  = 50
seq_length  = 50
data_loader = TextLoader(data_dir, batch_size, seq_length)

reading text file


In [10]:
## data_loader
print( "type of 'data_loader' is %s, length is %d" % (type(data_loader.vocab), len(data_loader.vocab)) )

type of 'data_loader' is <class 'dict'>, length is 75


In [11]:
## data_loader.vocab
print ("data_loader.vocab looks like \n%s " % (data_loader.vocab,))

data_loader.vocab looks like 
{'ᴥ': 0, ' ': 1, 'ㅇ': 2, 'ㅏ': 3, 'ㄴ': 4, 'ㄹ': 5, 'ㅣ': 6, 'ㄱ': 7, 'ㅡ': 8, 'ㅗ': 9, 'ㅅ': 10, 'ㅓ': 11, 'ㅁ': 12, 'ㄷ': 13, 'ㅎ': 14, 'ㅈ': 15, 'ㅜ': 16, 'ㅂ': 17, 'ㅕ': 18, '\n': 19, 'ㅔ': 20, 'ㅐ': 21, 'ㅆ': 22, 'ㅊ': 23, '.': 24, 'ㅢ': 25, 'ㅘ': 26, ',': 27, '"': 28, 'ㄲ': 29, 'ㅌ': 30, 'ㅍ': 31, 'ㄸ': 32, 'ㅚ': 33, 'ㅑ': 34, 'ㅟ': 35, 'ㅝ': 36, 'ㅠ': 37, 'ㅛ': 38, 'ㅖ': 39, 'ㅉ': 40, '?': 41, 'ㅋ': 42, 'ㅄ': 43, 'ㄶ': 44, '(': 45, ')': 46, 'ㅃ': 47, 'ㄺ': 48, "'": 49, 'ㄵ': 50, 'ㅀ': 51, '!': 52, 'ㅙ': 53, 'ㄼ': 54, 'ㄻ': 55, 'ㄿ': 56, ':': 57, '1': 58, '6': 59, '2': 60, 'ㄳ': 61, 'ㅞ': 62, '9': 63, '5': 64, '-': 65, '4': 66, '3': 67, '8': 68, '7': 69, '_': 70, 'ㄾ': 71, 'ㅒ': 72, '0': 73, '>': 74} 


In [12]:
## data_loader.chars
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )

type of 'data_loader.chars' is <class 'tuple'>, length is 75


In [13]:
## char index -> char
print ("data_loader.chars looks like \n%s " % (data_loader.chars,))

data_loader.chars looks like 
('ᴥ', ' ', 'ㅇ', 'ㅏ', 'ㄴ', 'ㄹ', 'ㅣ', 'ㄱ', 'ㅡ', 'ㅗ', 'ㅅ', 'ㅓ', 'ㅁ', 'ㄷ', 'ㅎ', 'ㅈ', 'ㅜ', 'ㅂ', 'ㅕ', '\n', 'ㅔ', 'ㅐ', 'ㅆ', 'ㅊ', '.', 'ㅢ', 'ㅘ', ',', '"', 'ㄲ', 'ㅌ', 'ㅍ', 'ㄸ', 'ㅚ', 'ㅑ', 'ㅟ', 'ㅝ', 'ㅠ', 'ㅛ', 'ㅖ', 'ㅉ', '?', 'ㅋ', 'ㅄ', 'ㄶ', '(', ')', 'ㅃ', 'ㄺ', "'", 'ㄵ', 'ㅀ', '!', 'ㅙ', 'ㄼ', 'ㄻ', 'ㄿ', ':', '1', '6', '2', 'ㄳ', 'ㅞ', '9', '5', '-', '4', '3', '8', '7', '_', 'ㄾ', 'ㅒ', '0', '>') 


In [14]:
for i, char in enumerate(data_loader.chars):
    # GET INDEX OF THE CHARACTER
    idx = data_loader.vocab[char]
    print ("[%02d] %03s (%02d)" 
           % (i, automata("".join(char)), idx))

[00]     (00)
[01]     (01)
[02]     (02)
[03]   ㅏ (03)
[04]     (04)
[05]     (05)
[06]   ㅣ (06)
[07]     (07)
[08]   ㅡ (08)
[09]   ㅗ (09)
[10]     (10)
[11]   ㅓ (11)
[12]     (12)
[13]     (13)
[14]     (14)
[15]     (15)
[16]   ㅜ (16)
[17]     (17)
[18]   ㅕ (18)
[19]   
 (19)
[20]   ㅔ (20)
[21]   ㅐ (21)
[22]     (22)
[23]     (23)
[24]   . (24)
[25]   ㅢ (25)
[26]   ㅘ (26)
[27]   , (27)
[28]   " (28)
[29]     (29)
[30]     (30)
[31]     (31)
[32]     (32)
[33]   ㅚ (33)
[34]   ㅑ (34)
[35]   ㅟ (35)
[36]   ㅝ (36)
[37]   ㅠ (37)
[38]   ㅛ (38)
[39]   ㅖ (39)
[40]     (40)
[41]   ? (41)
[42]     (42)
[43]   ㅄ (43)
[44]   ㄶ (44)
[45]   ( (45)
[46]   ) (46)
[47]     (47)
[48]   ㄺ (48)
[49]   ' (49)
[50]   ㄵ (50)
[51]   ㅀ (51)
[52]   ! (52)
[53]   ㅙ (53)
[54]   ㄼ (54)
[55]   ㄻ (55)
[56]   ㄿ (56)
[57]   : (57)
[58]   1 (58)
[59]   6 (59)
[60]   2 (60)
[61]   ㄳ (61)
[62]   ㅞ (62)
[63]   9 (63)
[64]   5 (64)
[65]   - (65)
[66]   4 (66)
[67]   3 (67)
[68]   8 (68)
[69]   7 (69)
[70]   _ (70)
[71]  